# Auxiliary Classifier GAN

ref. ODENA, Augustus; OLAH, Christopher; SHLENS, Jonathon. 
     Conditional image synthesis with auxiliary classifier gans. 
     In: Proceedings of the 34th International Conference on Machine Learning-Volume 70. JMLR. org, 2017. p. 2642-2651.
     https://arxiv.org/pdf/1610.09585.pdf%5D

In [25]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

Using TensorFlow backend.


# Generator

In [8]:
def build_generator(latent_dim, channels, num_classes):

    model = Sequential()

    model.add(Dense(128 * 7 * 7, activation="relu", input_dim=latent_dim))
    model.add(Reshape((7, 7, 128)))
    model.add(BatchNormalization(momentum=0.8))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=3, padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=3, padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(channels, kernel_size=3, padding='same'))
    model.add(Activation("tanh"))

    model.summary()

    noise = Input(shape=(latent_dim,))
    label = Input(shape=(1,), dtype='int32')
    label_embedding = Flatten()(Embedding(num_classes, latent_dim)(label))

    model_input = multiply([noise, label_embedding])
    img = model(model_input)

    return Model([noise, label], img)

# Discriminator

In [9]:
def build_discriminator(img_shape, num_classes):

    model = Sequential()

    model.add(Conv2D(16, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.summary()

    img = Input(shape=img_shape)

    # Extract feature representation
    features = model(img)

    # Determine validity and label of the image
    validity = Dense(1, activation="sigmoid")(features)
    label = Dense(num_classes, activation="softmax")(features)

    return Model(img, [validity, label])

# Autiliary functions

In [17]:
def sample_images(epoch, latent_dim, G):
    r, c = 10, 10
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    sampled_labels = np.array([num for _ in range(r) for num in range(c)])
    gen_imgs = G.predict([noise, sampled_labels])
    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt,:,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/%d.png" % epoch)
    plt.close()

# perform the training

In [23]:
def train(G, D, combined, epochs, latent_dim, batch_size=128, sample_interval=50):

    # Load the dataset
    (X_train, y_train), (_, _) = mnist.load_data()

    # Configure inputs
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3)
    y_train = y_train.reshape(-1, 1)

    # Adversarial ground truths
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------

        # Select a random batch of images
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        # Sample noise as generator input
        noise = np.random.normal(0, 1, (batch_size, latent_dim))

        # The labels of the digits that the generator tries to create an
        # image representation of
        sampled_labels = np.random.randint(0, 10, (batch_size, 1))

        # Generate a half batch of new images
        gen_imgs = G.predict([noise, sampled_labels])

        # Image labels. 0-9 
        img_labels = y_train[idx]

        # Train the discriminator
        d_loss_real = D.train_on_batch(imgs, [valid, img_labels])
        d_loss_fake = D.train_on_batch(gen_imgs, [fake, sampled_labels])
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # ---------------------
        #  Train Generator
        # ---------------------

        # Train the generator
        g_loss = combined.train_on_batch([noise, sampled_labels], [valid, sampled_labels])

        # Plot the progress
        print ("%d [D loss: %f, acc.: %.2f%%, op_acc: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[3], 100*d_loss[4], g_loss[0]))

        # If at save interval => save generated image samples
        if epoch % sample_interval == 0 or epoch == epochs - 1:
            save_model(G, D)
            sample_images(epoch, latent_dim, G)

In [30]:
def save(model, model_name, path="saved_model"):
    if not os.path.exists(path):
        os.makedirs(path)
    model_path = "{}/{}.json".format(path, model_name)
    weights_path = "{}/{}_weights.hdf5".format(path, model_name)
    options = {"file_arch": model_path,
                "file_weight": weights_path}
    json_string = model.to_json()
    open(options['file_arch'], 'w').write(json_string)
    model.save_weights(options['file_weight'])

In [19]:
def save_model(G, D):
    save(G, "generator")
    save(D, "discriminator")

# main()

In [3]:
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)
num_classes = 10
latent_dim = 100

In [4]:
# create optimizer
optimizer = Adam(0.0002, 0.5)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [5]:
# type of losses used in the GAN
losses = ['binary_crossentropy', 'sparse_categorical_crossentropy']

In [10]:
# Build and compile the discriminator
D = build_discriminator(img_shape, num_classes)
D.compile(loss=losses,
    optimizer=optimizer,
    metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 16)        160       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 32)          4640      
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 32)          0         
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 32)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 8, 8, 32)         

In [11]:
# Build the generator
G = build_generator(latent_dim, channels, num_classes)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 6272)              633472    
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 7, 7, 128)         512       
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 128)       147584    
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 128)       0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 14, 14, 128)      

In [12]:
# The generator takes noise and the target label as input
# and generates the corresponding digit of that label
noise = Input(shape=(latent_dim,))
label = Input(shape=(1,))
img = G([noise, label])

In [13]:
# For the combined model we will only train the generator
D.trainable = False

In [14]:
# The discriminator takes generated image as input and determines validity
# and the label of that image
valid, target_label = D(img)

In [15]:
# The combined model  (stacked generator and discriminator)
# Trains the generator to fool the discriminator
combined = Model([noise, label], [valid, target_label])
combined.compile(loss=losses,
    optimizer=optimizer)

## run

In [32]:
# epochs=14000
# epochs=8000
train(G, D, combined, 
      latent_dim=latent_dim, 
      epochs=epochs, 
      batch_size=32, sample_interval=200)

0 [D loss: 2.451509, acc.: 93.75%, op_acc: 20.31%] [G loss: 2.737927]
1 [D loss: 2.514468, acc.: 96.88%, op_acc: 20.31%] [G loss: 2.855221]
2 [D loss: 2.474411, acc.: 92.19%, op_acc: 21.88%] [G loss: 2.656236]
3 [D loss: 2.725640, acc.: 87.50%, op_acc: 17.19%] [G loss: 3.029039]
4 [D loss: 2.497117, acc.: 98.44%, op_acc: 21.88%] [G loss: 3.208701]
5 [D loss: 2.458739, acc.: 95.31%, op_acc: 21.88%] [G loss: 2.973763]
6 [D loss: 2.522685, acc.: 96.88%, op_acc: 17.19%] [G loss: 3.335631]
7 [D loss: 2.348210, acc.: 90.62%, op_acc: 32.81%] [G loss: 3.142118]
8 [D loss: 2.688333, acc.: 90.62%, op_acc: 12.50%] [G loss: 3.605253]
9 [D loss: 2.535077, acc.: 93.75%, op_acc: 23.44%] [G loss: 3.882790]
10 [D loss: 2.507432, acc.: 92.19%, op_acc: 23.44%] [G loss: 3.586366]
11 [D loss: 2.631939, acc.: 89.06%, op_acc: 21.88%] [G loss: 3.631742]
12 [D loss: 2.536930, acc.: 89.06%, op_acc: 20.31%] [G loss: 3.656847]
13 [D loss: 2.518475, acc.: 93.75%, op_acc: 26.56%] [G loss: 3.915666]
14 [D loss: 2.51

116 [D loss: 2.424013, acc.: 56.25%, op_acc: 40.62%] [G loss: 3.583321]
117 [D loss: 2.227857, acc.: 59.38%, op_acc: 56.25%] [G loss: 3.211426]
118 [D loss: 2.389380, acc.: 54.69%, op_acc: 45.31%] [G loss: 3.145219]
119 [D loss: 2.401752, acc.: 54.69%, op_acc: 42.19%] [G loss: 3.281041]
120 [D loss: 2.399249, acc.: 56.25%, op_acc: 42.19%] [G loss: 3.303886]
121 [D loss: 2.144140, acc.: 45.31%, op_acc: 50.00%] [G loss: 3.249362]
122 [D loss: 2.467181, acc.: 51.56%, op_acc: 42.19%] [G loss: 3.238620]
123 [D loss: 2.301019, acc.: 67.19%, op_acc: 51.56%] [G loss: 3.317372]
124 [D loss: 2.268270, acc.: 51.56%, op_acc: 43.75%] [G loss: 3.275862]
125 [D loss: 2.479044, acc.: 39.06%, op_acc: 42.19%] [G loss: 3.391428]
126 [D loss: 2.483999, acc.: 40.62%, op_acc: 42.19%] [G loss: 3.190622]
127 [D loss: 2.308022, acc.: 59.38%, op_acc: 46.88%] [G loss: 3.366154]
128 [D loss: 2.485788, acc.: 40.62%, op_acc: 42.19%] [G loss: 3.052684]
129 [D loss: 2.195984, acc.: 59.38%, op_acc: 50.00%] [G loss: 3.

230 [D loss: 1.286916, acc.: 90.62%, op_acc: 70.31%] [G loss: 3.972858]
231 [D loss: 1.507288, acc.: 85.94%, op_acc: 57.81%] [G loss: 3.858632]
232 [D loss: 1.547324, acc.: 90.62%, op_acc: 53.12%] [G loss: 4.257560]
233 [D loss: 1.500854, acc.: 93.75%, op_acc: 50.00%] [G loss: 3.500083]
234 [D loss: 1.447761, acc.: 95.31%, op_acc: 57.81%] [G loss: 3.918326]
235 [D loss: 1.510373, acc.: 93.75%, op_acc: 59.38%] [G loss: 3.639195]
236 [D loss: 1.592979, acc.: 84.38%, op_acc: 53.12%] [G loss: 3.947243]
237 [D loss: 1.636277, acc.: 87.50%, op_acc: 57.81%] [G loss: 3.686613]
238 [D loss: 1.487167, acc.: 90.62%, op_acc: 60.94%] [G loss: 4.001275]
239 [D loss: 1.434483, acc.: 87.50%, op_acc: 56.25%] [G loss: 3.412946]
240 [D loss: 1.342981, acc.: 93.75%, op_acc: 64.06%] [G loss: 3.053321]
241 [D loss: 1.581881, acc.: 73.44%, op_acc: 65.62%] [G loss: 3.292810]
242 [D loss: 1.468046, acc.: 87.50%, op_acc: 59.38%] [G loss: 3.532378]
243 [D loss: 1.495869, acc.: 89.06%, op_acc: 60.94%] [G loss: 3.

344 [D loss: 1.579852, acc.: 62.50%, op_acc: 64.06%] [G loss: 2.239426]
345 [D loss: 1.539597, acc.: 54.69%, op_acc: 73.44%] [G loss: 2.109682]
346 [D loss: 1.530542, acc.: 53.12%, op_acc: 73.44%] [G loss: 2.075929]
347 [D loss: 1.301095, acc.: 71.88%, op_acc: 75.00%] [G loss: 1.738575]
348 [D loss: 1.433620, acc.: 60.94%, op_acc: 73.44%] [G loss: 2.394208]
349 [D loss: 1.325015, acc.: 73.44%, op_acc: 67.19%] [G loss: 1.940514]
350 [D loss: 1.290285, acc.: 68.75%, op_acc: 76.56%] [G loss: 1.899443]
351 [D loss: 1.274249, acc.: 65.62%, op_acc: 71.88%] [G loss: 1.869601]
352 [D loss: 1.271448, acc.: 65.62%, op_acc: 73.44%] [G loss: 2.076161]
353 [D loss: 1.448617, acc.: 56.25%, op_acc: 75.00%] [G loss: 1.997255]
354 [D loss: 1.642980, acc.: 56.25%, op_acc: 67.19%] [G loss: 1.984356]
355 [D loss: 1.543276, acc.: 46.88%, op_acc: 71.88%] [G loss: 2.006821]
356 [D loss: 1.557301, acc.: 40.62%, op_acc: 75.00%] [G loss: 1.809147]
357 [D loss: 1.505664, acc.: 51.56%, op_acc: 78.12%] [G loss: 2.

458 [D loss: 0.970980, acc.: 71.88%, op_acc: 85.94%] [G loss: 1.302388]
459 [D loss: 0.919019, acc.: 79.69%, op_acc: 85.94%] [G loss: 1.326761]
460 [D loss: 1.006255, acc.: 62.50%, op_acc: 87.50%] [G loss: 1.508907]
461 [D loss: 1.148153, acc.: 65.62%, op_acc: 84.38%] [G loss: 1.408625]
462 [D loss: 0.949359, acc.: 79.69%, op_acc: 85.94%] [G loss: 1.661310]
463 [D loss: 1.199116, acc.: 73.44%, op_acc: 76.56%] [G loss: 1.848282]
464 [D loss: 1.202458, acc.: 65.62%, op_acc: 79.69%] [G loss: 1.506445]
465 [D loss: 1.126678, acc.: 73.44%, op_acc: 79.69%] [G loss: 1.487461]
466 [D loss: 0.941114, acc.: 71.88%, op_acc: 89.06%] [G loss: 1.468820]
467 [D loss: 1.281098, acc.: 46.88%, op_acc: 84.38%] [G loss: 1.568251]
468 [D loss: 0.930979, acc.: 71.88%, op_acc: 87.50%] [G loss: 1.171894]
469 [D loss: 1.157237, acc.: 60.94%, op_acc: 85.94%] [G loss: 1.513664]
470 [D loss: 1.134048, acc.: 57.81%, op_acc: 89.06%] [G loss: 1.427173]
471 [D loss: 1.070239, acc.: 51.56%, op_acc: 89.06%] [G loss: 1.

...

8674 [D loss: 0.717957, acc.: 51.56%, op_acc: 100.00%] [G loss: 0.740260]
8675 [D loss: 0.775609, acc.: 48.44%, op_acc: 98.44%] [G loss: 0.709148]
8676 [D loss: 0.714202, acc.: 57.81%, op_acc: 98.44%] [G loss: 0.739175]
8677 [D loss: 0.719849, acc.: 68.75%, op_acc: 96.88%] [G loss: 0.801189]
8678 [D loss: 0.877670, acc.: 42.19%, op_acc: 95.31%] [G loss: 0.787015]
8679 [D loss: 0.769908, acc.: 48.44%, op_acc: 96.88%] [G loss: 0.685326]
8680 [D loss: 0.830841, acc.: 53.12%, op_acc: 96.88%] [G loss: 0.711020]
8681 [D loss: 0.808370, acc.: 51.56%, op_acc: 96.88%] [G loss: 0.729862]
8682 [D loss: 0.798553, acc.: 50.00%, op_acc: 96.88%] [G loss: 0.733490]
8683 [D loss: 0.741957, acc.: 50.00%, op_acc: 100.00%] [G loss: 0.716797]
8684 [D loss: 0.724174, acc.: 45.31%, op_acc: 100.00%] [G loss: 0.705476]
8685 [D loss: 0.828777, acc.: 60.94%, op_acc: 95.31%] [G loss: 0.713239]
8686 [D loss: 0.748126, acc.: 57.81%, op_acc: 98.44%] [G loss: 0.706888]
8687 [D loss: 0.780141, acc.: 51.56%, op_acc: 96

8786 [D loss: 0.724833, acc.: 42.19%, op_acc: 100.00%] [G loss: 0.731500]
8787 [D loss: 0.702327, acc.: 59.38%, op_acc: 100.00%] [G loss: 0.702349]
8788 [D loss: 0.694850, acc.: 56.25%, op_acc: 100.00%] [G loss: 0.725210]
8789 [D loss: 0.728935, acc.: 56.25%, op_acc: 98.44%] [G loss: 0.724334]
8790 [D loss: 0.766577, acc.: 48.44%, op_acc: 98.44%] [G loss: 0.711215]
8791 [D loss: 0.709879, acc.: 53.12%, op_acc: 100.00%] [G loss: 0.754216]
8792 [D loss: 0.767743, acc.: 50.00%, op_acc: 98.44%] [G loss: 0.747468]
8793 [D loss: 0.709922, acc.: 54.69%, op_acc: 100.00%] [G loss: 0.801133]
8794 [D loss: 0.753591, acc.: 42.19%, op_acc: 100.00%] [G loss: 0.774458]
8795 [D loss: 0.706621, acc.: 53.12%, op_acc: 100.00%] [G loss: 0.751491]
8796 [D loss: 0.753236, acc.: 42.19%, op_acc: 98.44%] [G loss: 0.734362]
8797 [D loss: 0.769900, acc.: 46.88%, op_acc: 96.88%] [G loss: 0.736662]
8798 [D loss: 0.726135, acc.: 60.94%, op_acc: 98.44%] [G loss: 0.760336]
8799 [D loss: 0.688217, acc.: 51.56%, op_acc

8898 [D loss: 0.711116, acc.: 56.25%, op_acc: 100.00%] [G loss: 0.772357]
8899 [D loss: 0.727839, acc.: 45.31%, op_acc: 100.00%] [G loss: 0.793210]
8900 [D loss: 0.712555, acc.: 62.50%, op_acc: 98.44%] [G loss: 0.807640]
8901 [D loss: 0.809957, acc.: 43.75%, op_acc: 96.88%] [G loss: 0.756295]
8902 [D loss: 0.765231, acc.: 51.56%, op_acc: 98.44%] [G loss: 0.774758]
8903 [D loss: 0.747255, acc.: 51.56%, op_acc: 98.44%] [G loss: 0.745868]
8904 [D loss: 0.712724, acc.: 56.25%, op_acc: 100.00%] [G loss: 0.809860]
8905 [D loss: 0.742563, acc.: 46.88%, op_acc: 100.00%] [G loss: 0.761041]
8906 [D loss: 0.700806, acc.: 56.25%, op_acc: 98.44%] [G loss: 0.780445]
8907 [D loss: 0.766001, acc.: 42.19%, op_acc: 98.44%] [G loss: 0.733506]
8908 [D loss: 0.886066, acc.: 57.81%, op_acc: 98.44%] [G loss: 0.795704]
8909 [D loss: 0.685270, acc.: 54.69%, op_acc: 100.00%] [G loss: 0.733389]
8910 [D loss: 0.805294, acc.: 54.69%, op_acc: 96.88%] [G loss: 0.856053]
8911 [D loss: 0.852516, acc.: 48.44%, op_acc: 

9010 [D loss: 0.687801, acc.: 60.94%, op_acc: 98.44%] [G loss: 0.767788]
9011 [D loss: 0.816349, acc.: 54.69%, op_acc: 96.88%] [G loss: 0.751546]
9012 [D loss: 0.665834, acc.: 70.31%, op_acc: 100.00%] [G loss: 0.738602]
9013 [D loss: 0.738659, acc.: 53.12%, op_acc: 98.44%] [G loss: 0.728468]
9014 [D loss: 0.836516, acc.: 53.12%, op_acc: 96.88%] [G loss: 0.706862]
9015 [D loss: 0.788793, acc.: 51.56%, op_acc: 96.88%] [G loss: 0.768261]
9016 [D loss: 0.900081, acc.: 45.31%, op_acc: 96.88%] [G loss: 0.790845]
9017 [D loss: 0.726323, acc.: 46.88%, op_acc: 100.00%] [G loss: 0.753406]
9018 [D loss: 0.855004, acc.: 60.94%, op_acc: 95.31%] [G loss: 0.725214]
9019 [D loss: 0.724891, acc.: 60.94%, op_acc: 98.44%] [G loss: 0.699150]
9020 [D loss: 0.761713, acc.: 46.88%, op_acc: 96.88%] [G loss: 0.762848]
9021 [D loss: 0.794393, acc.: 45.31%, op_acc: 96.88%] [G loss: 0.771658]
9022 [D loss: 0.780593, acc.: 46.88%, op_acc: 98.44%] [G loss: 0.748223]
9023 [D loss: 0.741618, acc.: 42.19%, op_acc: 100

KeyboardInterrupt: 